In [2]:
import pandas as pd
import geopandas as gpd
import rioxarray
import math
import shapely

from shapely.geometry import Point, MultiPoint

In [11]:
sightlines = pd.read_pickle("my_results/method_1/d06_sightlines_geometries.pickle")
sightlines = sightlines[["start_point", "end_point", "sight_line_points"]]